In [3]:
####################################################################################################
# SES Prototype – Constellation Stress Index & Joint Risk Radar
#
# This notebook builds a synthetic "stress index" timeseries by combining:
#  - Capacity risk per beam (from Risk-aware Capacity Advisor)
#  - Jamming / interference anomaly score
#  - SLA breach windows
#  - Signal loss windows
#  - Beam handover anomalies
#
# It exports:
#  - artifacts_stress/stress_timeseries.csv
#  - artifacts_stress/stress_top_windows.csv
####################################################################################################

import os
from pathlib import Path
import numpy as np
import pandas as pd

# -------------------------------------------------------------------
# Step 1 – Mount Google Drive and set paths
# -------------------------------------------------------------------
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

BASE_DIR = Path("/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses")
ART_CAP = BASE_DIR / "artifacts_capacity"
ART_SLA = BASE_DIR / "artifacts_sla"
ART_SIG = BASE_DIR / "artifacts_signal_loss"
ART_HAND = BASE_DIR / "artifacts_handover"
ART_STRESS = BASE_DIR / "artifacts_stress"

ART_STRESS.mkdir(exist_ok=True)

print("BASE_DIR    :", BASE_DIR)
print("ART_CAP     :", ART_CAP)
print("ART_SLA     :", ART_SLA)
print("ART_SIG     :", ART_SIG)
print("ART_HAND    :", ART_HAND)
print("ART_STRESS  :", ART_STRESS)

# -------------------------------------------------------------------
# Step 2 – Helper functions
# -------------------------------------------------------------------
def load_csv(path, parse_dates=None):
    if path.exists():
        print(f"Loading: {path}")
        return pd.read_csv(path, parse_dates=parse_dates)
    else:
        print(f"[WARN] Missing file: {path}")
        return None

def min_max_norm(series):
    s = series.astype(float)
    mn, mx = s.min(), s.max()
    if np.isfinite(mn) and np.isfinite(mx) and mx > mn:
        return (s - mn) / (mx - mn)
    else:
        return pd.Series(0.0, index=s.index)


def indicator_from_intervals(times, intervals, pad_before="0min", pad_after="0min"):
    """
    times: DatetimeIndex
    intervals: list of (start, end) timestamps
    returns Series of 0/1 indicating stress around those intervals.
    """
    out = np.zeros(len(times), dtype=float)
    pad_before = pd.to_timedelta(pad_before)
    pad_after = pd.to_timedelta(pad_after)
    for (s, e) in intervals:
        if pd.isna(s) or pd.isna(e):
            continue
        s_pad = s - pad_before
        e_pad = e + pad_after
        mask = (times >= s_pad) & (times <= e_pad)
        out[mask] = 1.0
    return pd.Series(out, index=times)


# -------------------------------------------------------------------
# Step 3 – Load capacity-risk timeseries (base grid)
# -------------------------------------------------------------------
# From the Risk-aware Capacity Advisor notebook we expect:
# artifacts_capacity/capacity_risk_timeseries.csv
# with columns: time, beam_id, demand_forecast_mbps, capacity_mbps, risk_score

df_cap = load_csv(ART_CAP / "capacity_risk_timeseries.csv", parse_dates=["time"])
if df_cap is None:
    raise RuntimeError(
        "Capacity risk dataset not found. "
        "Run the Risk-aware Capacity Advisor notebook first "
        "so it saves artifacts_capacity/capacity_risk_timeseries.csv"
    )

df_cap = df_cap.copy()
df_cap["time"] = pd.to_datetime(df_cap["time"], utc=True)
df_cap = df_cap.sort_values(["beam_id", "time"])
print("Capacity risk shape:", df_cap.shape)
print(df_cap.head())

# Base grid for stress index: every (time, beam_id) present in df_cap
times = df_cap["time"]
beams = df_cap["beam_id"].unique()
print("Beams in capacity dataset:", beams)

# -------------------------------------------------------------------
# Step 4 – Capacity component (already 0–1 risk_score)
# -------------------------------------------------------------------
# The column name 'risk_score' was not found in df_cap. Using 'risk_index' instead.
if "risk_index" not in df_cap.columns:
    raise RuntimeError("capacity_risk_timeseries.csv must have a 'risk_index' column.")
cap_risk = min_max_norm(df_cap["risk_index"])
df_cap["capacity_risk"] = cap_risk

# -------------------------------------------------------------------
# Step 5 – Jamming / interference component (global anomaly score)
# -------------------------------------------------------------------
# We use ses_comm_anomalies.csv anomaly_score and broadcast to beams.
COMM_ANOM = BASE_DIR / "ses_comm_anomalies.csv"
df_comm = load_csv(COMM_ANOM, parse_dates=["time"])

if df_comm is not None and "anomaly_score" in df_comm.columns:
    df_comm = df_comm.copy()
    df_comm["time"] = pd.to_datetime(df_comm["time"], utc=True)
    df_comm = df_comm.sort_values("time")
    jam = df_comm.set_index("time")["anomaly_score"].astype(float)
    # Reindex to our grid times, interpolate, forward-fill, then min-max normalise
    jam_on_grid = jam.reindex(times).interpolate().ffill().fillna(0.0)
    jam_risk = min_max_norm(jam_on_grid)
else:
    print("[INFO] No jamming anomalies found – setting jamming_risk = 0.")
    jam_risk = pd.Series(0.0, index=df_cap.index)

# because df_cap has multiple beams per time, we need same length; times is same length as df_cap
df_cap["jamming_risk"] = jam_risk.values

# -------------------------------------------------------------------
# Step 6 – SLA breach component (interval-based)
# -------------------------------------------------------------------
SLA_EVENTS = ART_SLA / "sla_breach_events.csv"
df_sla = load_csv(SLA_EVENTS, parse_dates=["start", "end"])

if df_sla is not None and not df_sla.empty:
    df_sla = df_sla.copy()
    df_sla["start"] = pd.to_datetime(df_sla["start"], utc=True, errors="coerce")
    df_sla["end"] = pd.to_datetime(df_sla["end"], utc=True, errors="coerce")
    intervals_sla = list(zip(df_sla["start"], df_sla["end"]))
    # pad a bit before/after each breach to reflect early-warning and lingering impact
    sla_indicator = indicator_from_intervals(times, intervals_sla,
                                             pad_before="5min", pad_after="10min")
else:
    print("[INFO] No SLA breach events found – sla_risk = 0.")
    sla_indicator = pd.Series(0.0, index=times)

# repeat over beams (same value for each beam at a given time)
df_cap["sla_risk"] = sla_indicator.values

# -------------------------------------------------------------------
# Step 7 – Signal-loss component (interval-based)
# -------------------------------------------------------------------
SIG_EVENTS = ART_SIG / "signal_loss_events.csv"
df_sig = load_csv(SIG_EVENTS, parse_dates=["start_time", "end_time"])

if df_sig is not None and not df_sig.empty:
    df_sig = df_sig.copy()
    start_col, end_col = None, None
    for c in df_sig.columns:
        lc = c.lower()
        if "start" in lc and start_col is None:
            start_col = c
        if "end" in lc and end_col is None:
            end_col = c
    if start_col is not None and end_col is not None:
        df_sig[start_col] = pd.to_datetime(df_sig[start_col], utc=True, errors="coerce")
        df_sig[end_col] = pd.to_datetime(df_sig[end_col], utc=True, errors="coerce")
        intervals_sig = list(zip(df_sig[start_col], df_sig[end_col]))
        loss_indicator = indicator_from_intervals(times, intervals_sig,
                                                  pad_before="2min", pad_after="5min")
    else:
        print("[WARN] Could not find start/end columns in signal-loss events – loss_risk = 0.")
        loss_indicator = pd.Series(0.0, index=times)
else:
    print("[INFO] No signal-loss events found – loss_risk = 0.")
    loss_indicator = pd.Series(0.0, index=times)

df_cap["loss_risk"] = loss_indicator.values

# -------------------------------------------------------------------
# Step 8 – Beam-handover component (point events)
# -------------------------------------------------------------------
HAND_TABLE = ART_HAND / "handover_table.csv"
df_hand = load_csv(HAND_TABLE, parse_dates=["t"])

if df_hand is not None and not df_hand.empty:
    df_hand = df_hand.copy()
    if "t" in df_hand.columns:
        df_hand["t"] = pd.to_datetime(df_hand["t"], utc=True, errors="coerce")
    else:
        # fallback: try first datetime-like column
        dt_col = None
        for c in df_hand.columns:
            if "time" in c.lower() or "t" == c.lower():
                dt_col = c
                break
        if dt_col is None:
            raise RuntimeError("handover_table.csv has no time column.")
        df_hand["t"] = pd.to_datetime(df_hand[dt_col], utc=True, errors="coerce")

    # treat anomalous == 1 as risky; if no such col, treat all as risky
    if "anomalous" in df_hand.columns:
        df_hand = df_hand[df_hand["anomalous"] == 1]

    intervals_hand = list(zip(df_hand["t"], df_hand["t"]))
    hand_indicator = indicator_from_intervals(times, intervals_hand,
                                              pad_before="2min", pad_after="2min")
else:
    print("[INFO] No beam handover anomalies found – handover_risk = 0.")
    hand_indicator = pd.Series(0.0, index=times)

df_cap["handover_risk"] = hand_indicator.values

# -------------------------------------------------------------------
# Step 9 – Compute stress index as weighted combination
# -------------------------------------------------------------------
# All components are in [0,1] (or 0/1). We now define a simple
# explainable linear formula (your "invention" – can be tuned later).

W_CAP = 0.40  # capacity risk is main driver
W_JAM = 0.20  # interference matters a lot
W_SLA = 0.15  # SLA breaches are important
W_LOSS = 0.15  # direct signal loss
W_HAND = 0.10  # handover risk

df_cap["stress_index_raw"] = (
    W_CAP * df_cap["capacity_risk"]
    + W_JAM * df_cap["jamming_risk"]
    + W_SLA * df_cap["sla_risk"]
    + W_LOSS * df_cap["loss_risk"]
    + W_HAND * df_cap["handover_risk"]
)

df_cap["stress_index"] = df_cap["stress_index_raw"].clip(0, 1)

print("Stress index stats:")
print(df_cap["stress_index"].describe())

# -------------------------------------------------------------------
# Step 10 – Export stress timeseries & top windows
# -------------------------------------------------------------------
cols_out = [
    "time",
    "beam_id",
    "capacity_risk",
    "jamming_risk",
    "sla_risk",
    "loss_risk",
    "handover_risk",
    "stress_index",
]

df_stress = df_cap[cols_out].copy()
df_stress.to_csv(ART_STRESS / "stress_timeseries.csv", index=False)
print("Saved:", ART_STRESS / "stress_timeseries.csv")

# Top windows per beam
df_top = (
    df_stress.sort_values("stress_index", ascending=False)
    .groupby("beam_id")
    .head(10)
    .reset_index(drop=True)
)
df_top.to_csv(ART_STRESS / "stress_top_windows.csv", index=False)
print("Saved:", ART_STRESS / "stress_top_windows.csv")

# Simple global averages (can be used as a "feature importance" approximation)
df_contrib = df_stress[[
    "capacity_risk",
    "jamming_risk",
    "sla_risk",
    "loss_risk",
    "handover_risk",
    "stress_index",
]].mean().to_frame(name="global_mean").reset_index().rename(columns={"index": "component"})
df_contrib.to_csv(ART_STRESS / "stress_global_means.csv", index=False)
print("Saved:", ART_STRESS / "stress_global_means.csv")

print("\n--- DONE: Constellation Stress Index built. ---")

Mounted at /content/drive
BASE_DIR    : /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses
ART_CAP     : /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_capacity
ART_SLA     : /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_sla
ART_SIG     : /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_signal_loss
ART_HAND    : /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_handover
ART_STRESS  : /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_stress
Loading: /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/artifacts_capacity/capacity_risk_timeseries.csv
Capacity risk shape: (25168, 12)
                        time beam_id  demand_mbps  capacity_mbps  demand_norm  \
2  2021-10-18 07:20:00+00:00  AFRICA   155.229844            700     0.221757   
6  2021-10-18 07:30:00+00:00  AFRICA   125.468461            700     0.179241   
10 2021-10-18 07:40:00+00:00  AFRICA   143.945